In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 사용 패키지

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
import datetime

# 데이터 로드

In [3]:
path = "/content/drive/MyDrive/Colab Notebooks/cw_daicon/gas/"

In [4]:
total = pd.read_csv(path+'20181231.csv', encoding='cp949')

In [5]:
total.head()

,연월일,시간,구분,공급량
0,2013-01-01,1,A,2497.129
1,2013-01-01,2,A,2363.265
2,2013-01-01,3,A,2258.505
3,2013-01-01,4,A,2243.969
4,2013-01-01,5,A,2344.105


# 전처리

In [6]:
total['구분'].unique()

array(['A', 'B', 'C', 'D', 'E', 'G', 'H'], dtype=object)

In [7]:
d_map = {}
for i, d in enumerate(total['구분'].unique()):
    d_map[d] = i
total['구분'] = total['구분'].map(d_map)

In [8]:
total['연월일'] = pd.to_datetime(total['연월일'])

In [9]:
total['year'] = total['연월일'].dt.year
total['month'] = total['연월일'].dt.month
total['day'] = total['연월일'].dt.day
total['weekday'] = total['연월일'].dt.weekday

In [10]:
total

,연월일,시간,구분,공급량,year,month,day,weekday
0,2013-01-01,1,0,2497.129,2013,1,1,1
1,2013-01-01,2,0,2363.265,2013,1,1,1
2,2013-01-01,3,0,2258.505,2013,1,1,1
3,2013-01-01,4,0,2243.969,2013,1,1,1
4,2013-01-01,5,0,2344.105,2013,1,1,1
...,...,...,...,...,...,...,...,...
368083,2018-12-31,20,6,681.033,2018,12,31,0
368084,2018-12-31,21,6,669.961,2018,12,31,0
368085,2018-12-31,22,6,657.941,2018,12,31,0
368086,2018-12-31,23,6,610.953,2018,12,31,0


In [22]:
total

,연월일,시간,구분,공급량,year,month,day,weekday
0,2013-01-01,1,0,2497.129,2013,1,1,1
1,2013-01-01,2,0,2363.265,2013,1,1,1
2,2013-01-01,3,0,2258.505,2013,1,1,1
3,2013-01-01,4,0,2243.969,2013,1,1,1
4,2013-01-01,5,0,2344.105,2013,1,1,1
...,...,...,...,...,...,...,...,...
368083,2018-12-31,20,6,681.033,2018,12,31,0
368084,2018-12-31,21,6,669.961,2018,12,31,0
368085,2018-12-31,22,6,657.941,2018,12,31,0
368086,2018-12-31,23,6,610.953,2018,12,31,0


## Feature Engineering

In [30]:
# 전년도 가스 공급량 feature 추가
def last_year(x) :
    alp = x['구분']
    day = x['연월일']
    return day-datetime.timedelta(days=1)

total['last_y'] = total.apply(last_year, axis=1)
total

,연월일,시간,구분,공급량,year,month,day,weekday,last_y
0,2013-01-01,1,0,2497.129,2013,1,1,1,2012-12-31
1,2013-01-01,2,0,2363.265,2013,1,1,1,2012-12-31
2,2013-01-01,3,0,2258.505,2013,1,1,1,2012-12-31
3,2013-01-01,4,0,2243.969,2013,1,1,1,2012-12-31
4,2013-01-01,5,0,2344.105,2013,1,1,1,2012-12-31
...,...,...,...,...,...,...,...,...,...
368083,2018-12-31,20,6,681.033,2018,12,31,0,2018-12-30
368084,2018-12-31,21,6,669.961,2018,12,31,0,2018-12-30
368085,2018-12-31,22,6,657.941,2018,12,31,0,2018-12-30
368086,2018-12-31,23,6,610.953,2018,12,31,0,2018-12-30


In [33]:
# def last_gas(x):
#     if str(x['last_y'])[:4] == '2012':
#         return -1
#     day = x['last_y']
    
#     gas = x[x['연월일'] == day]['공급량']
#     return gas

# total['last_gas'] =  total.apply(lambda, axis=1)
# total

KeyError: ignored

In [35]:
# tt = total.copy()

In [41]:
# tt['last_gas'] = tt.groupby('구분')['공급량'].shift(8760)
# tt

,연월일,시간,구분,공급량,year,month,day,weekday,last_y,last_gas
0,2013-01-01,1,0,2497.129,2013,1,1,1,2012-12-31,NaN
1,2013-01-01,2,0,2363.265,2013,1,1,1,2012-12-31,NaN
2,2013-01-01,3,0,2258.505,2013,1,1,1,2012-12-31,NaN
3,2013-01-01,4,0,2243.969,2013,1,1,1,2012-12-31,NaN
4,2013-01-01,5,0,2344.105,2013,1,1,1,2012-12-31,NaN
...,...,...,...,...,...,...,...,...,...,...
368083,2018-12-31,20,6,681.033,2018,12,31,0,2018-12-30,517.264
368084,2018-12-31,21,6,669.961,2018,12,31,0,2018-12-30,530.896
368085,2018-12-31,22,6,657.941,2018,12,31,0,2018-12-30,506.287
368086,2018-12-31,23,6,610.953,2018,12,31,0,2018-12-30,470.638


In [37]:
# tt[tt['구분'] == 1]

,연월일,시간,구분,공급량,year,month,day,weekday,last_y,last_gas
8760,2013-01-01,1,1,2169.093,2013,1,1,1,2012-12-31,NaN
8761,2013-01-01,2,1,2051.733,2013,1,1,1,2012-12-31,NaN
8762,2013-01-01,3,1,1923.253,2013,1,1,1,2012-12-31,NaN
8763,2013-01-01,4,1,1927.253,2013,1,1,1,2012-12-31,NaN
8764,2013-01-01,5,1,2047.333,2013,1,1,1,2012-12-31,NaN
...,...,...,...,...,...,...,...,...,...,...
324283,2018-12-31,20,1,2668.688,2018,12,31,0,2018-12-30,1800.603
324284,2018-12-31,21,1,2606.681,2018,12,31,0,2018-12-30,1735.438
324285,2018-12-31,22,1,2540.169,2018,12,31,0,2018-12-30,1765.749
324286,2018-12-31,23,1,2407.945,2018,12,31,0,2018-12-30,1980.802


In [42]:
# tt[tt['구분'] == 1][:1]

,연월일,시간,구분,공급량,year,month,day,weekday,last_y,last_gas
8760,2013-01-01,1,1,2169.093,2013,1,1,1,2012-12-31,NaN


In [43]:
# tt[tt['구분'] == 1][8760:8761]
# tt[tt['구분'] == 1][17520:17521]
# tt[tt['구분'] == 1][26280:26281]
# tt[tt['구분'] == 1][35040:35041]
# tt[tt['구분'] == 1][43801:43801]

,연월일,시간,구분,공급량,year,month,day,weekday,last_y,last_gas
70080,2014-01-01,1,1,1468.293,2014,1,1,2,2013-12-31,2169.093


In [73]:
total = total.drop('last_y',axis=1)
total

,연월일,시간,구분,공급량,year,month,day,weekday,last_gas
0,2013-01-01,1,0,2497.129,2013,1,1,1,NaN
1,2013-01-01,2,0,2363.265,2013,1,1,1,NaN
2,2013-01-01,3,0,2258.505,2013,1,1,1,NaN
3,2013-01-01,4,0,2243.969,2013,1,1,1,NaN
4,2013-01-01,5,0,2344.105,2013,1,1,1,NaN
...,...,...,...,...,...,...,...,...,...
368083,2018-12-31,20,6,681.033,2018,12,31,0,517.264
368084,2018-12-31,21,6,669.961,2018,12,31,0,530.896
368085,2018-12-31,22,6,657.941,2018,12,31,0,506.287
368086,2018-12-31,23,6,610.953,2018,12,31,0,470.638


In [48]:
total['last_gas'] = total.groupby('구분')['공급량'].shift(8760)
total

,연월일,시간,구분,공급량,year,month,day,weekday,last_y,last_gas
0,2013-01-01,1,0,2497.129,2013,1,1,1,2012-12-31,NaN
1,2013-01-01,2,0,2363.265,2013,1,1,1,2012-12-31,NaN
2,2013-01-01,3,0,2258.505,2013,1,1,1,2012-12-31,NaN
3,2013-01-01,4,0,2243.969,2013,1,1,1,2012-12-31,NaN
4,2013-01-01,5,0,2344.105,2013,1,1,1,2012-12-31,NaN
...,...,...,...,...,...,...,...,...,...,...
368083,2018-12-31,20,6,681.033,2018,12,31,0,2018-12-30,517.264
368084,2018-12-31,21,6,669.961,2018,12,31,0,2018-12-30,530.896
368085,2018-12-31,22,6,657.941,2018,12,31,0,2018-12-30,506.287
368086,2018-12-31,23,6,610.953,2018,12,31,0,2018-12-30,470.638


In [50]:
train_years = [2014,2015,2016,2017]  #2013은 2012년도가 없어서 예외
val_years = [2018]

In [51]:
train = total[total['year'].isin(train_years)]
val = total[total['year'].isin(val_years)]

In [52]:
features = ['구분', 'month', 'day', 'weekday', '시간','last_gas']
train_x = train[features]
train_y = train['공급량']

val_x = val[features]
val_y = val['공급량']

In [53]:
train_x

,구분,month,day,weekday,시간,last_gas
61320,0,1,1,2,1,2497.129
61321,0,1,1,2,2,2363.265
61322,0,1,1,2,3,2258.505
61323,0,1,1,2,4,2243.969
61324,0,1,1,2,5,2344.105
...,...,...,...,...,...,...
306763,6,12,31,6,20,531.796
306764,6,12,31,6,21,504.268
306765,6,12,31,6,22,482.476
306766,6,12,31,6,23,450.284


# 학습

In [54]:
d_train = lgb.Dataset(train_x, train_y)
d_val = lgb.Dataset(val_x, val_y)

params = {
    'objective': 'regression',
    'metric':'mae',
    'seed':42
}

model = lgb.train(params, d_train, 500, d_val, verbose_eval=20, early_stopping_rounds=10)

Training until validation scores don't improve for 10 rounds.
[20]	valid_0's l1: 206.153
[40]	valid_0's l1: 170.528
[60]	valid_0's l1: 166.578
[80]	valid_0's l1: 165.465
[100]	valid_0's l1: 164.219
[120]	valid_0's l1: 163.117
[140]	valid_0's l1: 162.712
Early stopping, best iteration is:
[145]	valid_0's l1: 162.399


# 추론 및 결과 제출

In [64]:
test = pd.read_csv(path+'test.csv')
submission = pd.read_csv(path+'sample_submission.csv')

In [65]:
test.head()

,일자|시간|구분
0,2019-01-01 01 A
1,2019-01-01 02 A
2,2019-01-01 03 A
3,2019-01-01 04 A
4,2019-01-01 05 A


In [66]:
submission.head()

,일자|시간|구분,공급량
0,2019-01-01 01 A,0
1,2019-01-01 02 A,0
2,2019-01-01 03 A,0
3,2019-01-01 04 A,0
4,2019-01-01 05 A,0


In [67]:
test['일자'] = test['일자|시간|구분'].str.split(' ').str[0]
test['시간'] = test['일자|시간|구분'].str.split(' ').str[1].astype(int)
test['구분'] = test['일자|시간|구분'].str.split(' ').str[2]

In [68]:
test['일자'] = pd.to_datetime(test['일자'])
test['year'] = test['일자'].dt.year
test['month'] = test['일자'].dt.month
test['day'] = test['일자'].dt.day
test['weekday'] = test['일자'].dt.weekday

In [69]:
test['구분'] = test['구분'].map(d_map)

In [72]:
total.head()

,연월일,시간,구분,공급량,year,month,day,weekday,last_y,last_gas
0,2013-01-01,1,0,2497.129,2013,1,1,1,2012-12-31,NaN
1,2013-01-01,2,0,2363.265,2013,1,1,1,2012-12-31,NaN
2,2013-01-01,3,0,2258.505,2013,1,1,1,2012-12-31,NaN
3,2013-01-01,4,0,2243.969,2013,1,1,1,2012-12-31,NaN
4,2013-01-01,5,0,2344.105,2013,1,1,1,2012-12-31,NaN


In [79]:
total_drop = total.drop('last_gas',axis=1)
total_drop

,연월일,시간,구분,공급량,year,month,day,weekday
0,2013-01-01,1,0,2497.129,2013,1,1,1
1,2013-01-01,2,0,2363.265,2013,1,1,1
2,2013-01-01,3,0,2258.505,2013,1,1,1
3,2013-01-01,4,0,2243.969,2013,1,1,1
4,2013-01-01,5,0,2344.105,2013,1,1,1
...,...,...,...,...,...,...,...,...
368083,2018-12-31,20,6,681.033,2018,12,31,0
368084,2018-12-31,21,6,669.961,2018,12,31,0
368085,2018-12-31,22,6,657.941,2018,12,31,0
368086,2018-12-31,23,6,610.953,2018,12,31,0


In [84]:
total_drop['공급량_x'] = total_drop['공급량']
total_drop = total_drop.drop('공급량',axis=1)
total_drop.rename(columns = {'공급량_x' : '공급량'}, inplace = True)
total_drop

,연월일,시간,구분,year,month,day,weekday,공급량
0,2013-01-01,1,0,2013,1,1,1,2497.129
1,2013-01-01,2,0,2013,1,1,1,2363.265
2,2013-01-01,3,0,2013,1,1,1,2258.505
3,2013-01-01,4,0,2013,1,1,1,2243.969
4,2013-01-01,5,0,2013,1,1,1,2344.105
...,...,...,...,...,...,...,...,...
368083,2018-12-31,20,6,2018,12,31,0,681.033
368084,2018-12-31,21,6,2018,12,31,0,669.961
368085,2018-12-31,22,6,2018,12,31,0,657.941
368086,2018-12-31,23,6,2018,12,31,0,610.953


In [81]:
test_lag = test.drop('일자|시간|구분', axis=1)
test_lag.rename(columns = {'일자' : '연월일'}, inplace = True)
test_lag['공급량'] = ''

test_lag

,연월일,시간,구분,year,month,day,weekday,공급량
0,2019-01-01,1,0,2019,1,1,1,
1,2019-01-01,2,0,2019,1,1,1,
2,2019-01-01,3,0,2019,1,1,1,
3,2019-01-01,4,0,2019,1,1,1,
4,2019-01-01,5,0,2019,1,1,1,
...,...,...,...,...,...,...,...,...
15115,2019-03-31,20,6,2019,3,31,6,
15116,2019-03-31,21,6,2019,3,31,6,
15117,2019-03-31,22,6,2019,3,31,6,
15118,2019-03-31,23,6,2019,3,31,6,


In [86]:
test_2 = pd.concat([total_drop, test_lag], axis=0)

In [87]:
test_2['last_gas'] = test_2.groupby('구분')['공급량'].shift(8760)
test_2

,연월일,시간,구분,year,month,day,weekday,공급량,last_gas
0,2013-01-01,1,0,2013,1,1,1,2497.13,NaN
1,2013-01-01,2,0,2013,1,1,1,2363.26,NaN
2,2013-01-01,3,0,2013,1,1,1,2258.51,NaN
3,2013-01-01,4,0,2013,1,1,1,2243.97,NaN
4,2013-01-01,5,0,2013,1,1,1,2344.11,NaN
...,...,...,...,...,...,...,...,...,...
15115,2019-03-31,20,6,2019,3,31,6,,244.162
15116,2019-03-31,21,6,2019,3,31,6,,248.059
15117,2019-03-31,22,6,2019,3,31,6,,231.181
15118,2019-03-31,23,6,2019,3,31,6,,199.022


In [91]:
test_3 = test_2[len(total_drop):]

In [108]:
test_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15120 entries, 0 to 15119
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   연월일       15120 non-null  datetime64[ns]
 1   시간        15120 non-null  int64         
 2   구분        15120 non-null  int64         
 3   year      15120 non-null  int64         
 4   month     15120 non-null  int64         
 5   day       15120 non-null  int64         
 6   weekday   15120 non-null  int64         
 7   공급량       15120 non-null  object        
 8   last_gas  15120 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(6), object(1)
memory usage: 1.2+ MB


In [107]:
test_3['last_gas'] = test_3['last_gas'].astype(float)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [109]:
test_x = test_3[features]

In [110]:
test_x

,구분,month,day,weekday,시간,last_gas
0,0,1,1,1,1,1765.008
1,0,1,1,1,2,1679.186
2,0,1,1,1,3,1610.885
3,0,1,1,1,4,1604.123
4,0,1,1,1,5,1711.506
...,...,...,...,...,...,...
15115,6,3,31,6,20,244.162
15116,6,3,31,6,21,248.059
15117,6,3,31,6,22,231.181
15118,6,3,31,6,23,199.022


In [111]:
preds = model.predict(test_x)

In [112]:
submission['공급량'] = preds

In [113]:
submission

,일자|시간|구분,공급량
0,2019-01-01 01 A,1816.842154
1,2019-01-01 02 A,1615.836707
2,2019-01-01 03 A,1579.566791
3,2019-01-01 04 A,1579.566791
4,2019-01-01 05 A,1705.998810
...,...,...
15115,2019-03-31 20 H,262.506313
15116,2019-03-31 21 H,262.506313
15117,2019-03-31 22 H,245.458270
15118,2019-03-31 23 H,212.147833


In [114]:
submission.to_csv('submission2.csv', index=False)